In [1]:
import numpy as np
import RedLionfishDeconv.RLDeconv3DScipy as rls
import tifffile
import os
import napari
import logging

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
#Path is referenced from notebook location
data = tifffile.imread("testdata/gendata_psfconv_poiss.tif")
psf = tifffile.imread("testdata/PSF_RFI_8bit.tif")

In [4]:
data.shape

(160, 160, 160)

In [5]:
psf.sum()

1094998

In [6]:
psf_f = psf.astype(np.float32)

In [7]:
psf_norm = psf_f/psf_f.sum()

In [8]:
psf_norm.sum()

1.0

In [9]:
print("Starting RL calculation using doRLDeconvolution_DL2_4()")
res0 = rls.doRLDeconvolution_DL2_4(data,psf_norm, niter=3)
print("Completed")
print(f"res0 shape:{res0.shape}, dtype: {res0.dtype}, max,min: {res0.max()},{res0.min()}")

INFO:root:doRLDeconvolution_DL2_4() (CPU)
INFO:root:change3DSizeTo() , shape:(160, 160, 160)


Starting RL calculation using doRLDeconvolution_DL2_4()
Completed
res0 shape:(160, 160, 160), dtype: float32, max,min: 1293.83447265625,0.0


OpenCL - default platform (CUDA)

In [10]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()")
res_ocl_cuda = rlocl.nonBlock_RLDeconvolutionReiknaOCL(data,psf_norm, niter=3)
print("Completed")
print(f"res1 shape:{res_ocl_cuda.shape}, dtype: {res_ocl_cuda.dtype}, max,min: {res_ocl_cuda.max()},{res_ocl_cuda.min()}")

INFO:root:nonBlock_RLDeconvolutionReiknaOCL(), niter:3, platform_preference:cuda


Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()


INFO:root:OCL device selected : <pyopencl.Device 'NVIDIA GeForce GTX 1650 with Max-Q Design' on 'NVIDIA CUDA' at 0x146c325a6b0>
INFO:root:Device params max_work_item_sizes: [1024, 1024, 64]
INFO:root:setPSF()
INFO:root:change3DSizeTo() , shape:(160, 160, 160)
INFO:root:doRLDeconvolution()
INFO:root:RL completed, result collected
INFO:root:Clearing GPU RAM


Completed
res1 shape:(160, 160, 160), dtype: float32, max,min: 1293.833984375,0.0


opencl Intel CPU

In [11]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()")
rlocl.RL_CL_PLATFORM_PREFERENCE="intel"
res_ocl_intel = rlocl.nonBlock_RLDeconvolutionReiknaOCL(data,psf_norm, niter=3)
print("Completed")
print(f"res1 shape:{res_ocl_intel.shape}, dtype: {res_ocl_intel.dtype}, max,min: {res_ocl_intel.max()},{res_ocl_intel.min()}")

INFO:root:nonBlock_RLDeconvolutionReiknaOCL(), niter:3, platform_preference:intel
INFO:root:OCL device selected : <pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x146c36e1d40>
INFO:root:Device params max_work_item_sizes: [512, 512, 512]


Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()


INFO:root:setPSF()
INFO:root:change3DSizeTo() , shape:(160, 160, 160)
INFO:root:doRLDeconvolution()
INFO:root:RL completed, result collected
INFO:root:Clearing GPU RAM


Completed
res1 shape:(160, 160, 160), dtype: float32, max,min: 1293.833984375,0.0


In [12]:
import skimage.restoration
print("Starting RL calculation using skimage.restoration.richardson_lucy()")
res_skimage = skimage.restoration.richardson_lucy(data,psf_norm, num_iter = 3, clip=False)
print("Completed")
print(f"res_skimage shape:{res_skimage.shape}, dtype: {res_skimage.dtype}, max,min: {res_skimage.max()},{res_skimage.min()}")

Starting RL calculation using skimage.restoration.richardson_lucy()
Completed
res_skimage shape:(160, 160, 160), dtype: float64, max,min: 283.66256433703546,-1.7658841937650016e-45


Strangely they give different results in intensity, so maybe not a good way to test RL code

In [13]:
NV=napari.Viewer()
NV.add_image(res0)
NV.add_image(res_ocl_cuda)
NV.add_image(res_skimage)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Assistant skips harvesting pyclesperanto as it's not installed.


<Image layer 'res_skimage' at 0x146dfb84dd0>

# Compare outputs

In [14]:
# absolute(a - b) <= (atol + rtol * absolute(b))

#np.allclose(res0,res1,rtol=1e-5, atol=1e-30)
#np.allclose(res0,res1,atol=1e-30)
np.allclose(res0,res_ocl_cuda) #Use defaule rtol and atol 

True

In [15]:
np.allclose(res0,res_ocl_intel) #Use defaule rtol and atol 

True

In [16]:
np.allclose(res_ocl_cuda,res_ocl_intel) #Use defaule rtol and atol

True

Some diffeerences but it is small

In [17]:
#Maximum difference
np.max(np.abs(res0-res_ocl_cuda))

0.00091552734

In [18]:
#Maximum difference
np.max(np.abs(res0-res_skimage))

1035.6353393268591

In [19]:
#Just for curiosity, compare the sum of values before and after convolution
print(np.sum(data))
print(np.sum(res0))
print(np.sum(res_ocl_cuda))
print(np.sum(res_skimage))

18409407
18559448.0
18559448.0
18626902.969301894


## Large(r) data

In [20]:
#Generate 256x256x256 data for testing
#Create a 2d array to do FFT
ashape = (256,256,256)

a = np.zeros(ashape, dtype=np.float32)

#Add a few cubes in grid-like locations
cubesize=2
cubespacing=60
for iz in range(int(cubespacing/2),ashape[0],cubespacing):
    for iy in range(int(cubespacing/2),ashape[1],cubespacing):
        for ix in range(int(cubespacing/2),ashape[2],cubespacing):
            a[iz:iz+cubesize , iy:iy+cubesize , ix:ix+cubesize] = np.ones((cubesize,cubesize,cubesize))

#Convolute with experimental PSF
psf = tifffile.imread("testdata/PSF_RFI_8bit.tif")
psf_f = psf.astype(np.float32)
psf_norm = psf_f/ np.sum(psf_f)
assert np.sum(psf_norm)==1.0

import scipy.signal
data_large = scipy.signal.convolve(a, psf_norm, mode='same')
print(data_large.shape)

(256, 256, 256)


In [21]:
print("Starting RL calculation using doRLDeconvolution_DL2_4()")
res0 = rls.doRLDeconvolution_DL2_4(data_large,psf_norm, niter=3)
print("Completed")
print(f"res0 shape:{res0.shape}, dtype: {res0.dtype}, max,min: {res0.max()},{res0.min()}")

INFO:root:doRLDeconvolution_DL2_4() (CPU)
INFO:root:change3DSizeTo() , shape:(256, 256, 256)


Starting RL calculation using doRLDeconvolution_DL2_4()
Completed
res0 shape:(256, 256, 256), dtype: float32, max,min: 0.009700462222099304,-8.700173566467129e-06


In [22]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()")
res_ocl_cuda = rlocl.nonBlock_RLDeconvolutionReiknaOCL(data_large,psf_norm, niter=3)
print("Completed")
print(f"res1 shape:{res_ocl_cuda.shape}, dtype: {res_ocl_cuda.dtype}, max,min: {res_ocl_cuda.max()},{res_ocl_cuda.min()}")

INFO:root:nonBlock_RLDeconvolutionReiknaOCL(), niter:3, platform_preference:intel
INFO:root:OCL device selected : <pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x146c36e1d40>
INFO:root:Device params max_work_item_sizes: [512, 512, 512]


Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()


INFO:root:setPSF()
INFO:root:change3DSizeTo() , shape:(256, 256, 256)
INFO:root:doRLDeconvolution()
INFO:root:RL completed, result collected
INFO:root:Clearing GPU RAM


Completed
res1 shape:(256, 256, 256), dtype: float32, max,min: 0.009696180000901222,0.0


In [23]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using block_RLDeconv3DReiknaOCL()")
res2 = rlocl.block_RLDeconv3DReiknaOCL(data_large,psf_norm,niter=3, max_dim_size = 128)
print("Completed")
print(f"res2 shape:{res2.shape}, dtype: {res2.dtype}, max,min: {res2.max()},{res2.min()}")

INFO:root:block_RLDeconv3DReiknaOCL() , data.shape:(256, 256, 256), psfdata.shape:(61, 32, 32), max_dim_size:128, psfpaddingfract:1.2
INFO:root:data shape: (256, 256, 256) , psf shape: (61, 32, 32)
INFO:root:blockshape: [128, 128, 128]
INFO:root:blockstep: [54, 89, 89]
INFO:root:OCL device selected : <pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x146c36e1d40>


Starting RL calculation using block_RLDeconv3DReiknaOCL()


INFO:root:Device params max_work_item_sizes: [512, 512, 512]
INFO:root:setPSF()
INFO:root:change3DSizeTo() , shape:[128, 128, 128]
INFO:root:Beggining block-by-block iterations.
INFO:root:New block, intended origin iz0,iy0,ix0 = 0,0,0 , use origin iz00,iy00,ix00 = 0,0,0 , end iz1,iy1,ix1 = 128,128,128
INFO:root:Start RL deconvolution of this block
INFO:root:doRLDeconvolution()
INFO:root:RL completed, result collected
INFO:root:Clearing GPU RAM
INFO:root:This block's RL deconvolution completed
INFO:root:Crop block result from origin jz0,jy0,jx0 = : 0,0,0
INFO:root:Copying cropped block to datares
INFO:root:New block, intended origin iz0,iy0,ix0 = 0,0,89 , use origin iz00,iy00,ix00 = 0,0,89 , end iz1,iy1,ix1 = 128,128,217
INFO:root:Start RL deconvolution of this block
INFO:root:doRLDeconvolution()
INFO:root:RL completed, result collected
INFO:root:Clearing GPU RAM
INFO:root:This block's RL deconvolution completed
INFO:root:Crop block result from origin jz0,jy0,jx0 = : 0,0,19
INFO:root:Co

Completed
res2 shape:(256, 256, 256), dtype: float32, max,min: 0.00954804103821516,0.0


In [24]:
import skimage.restoration
print("Starting RL calculation using skimage.restoration.richardson_lucy()")
res_skimage = skimage.restoration.richardson_lucy(data_large,psf_norm, num_iter = 3, clip=False)
print("Completed")
print(f"res_skimage shape:{res_skimage.shape}, dtype: {res_skimage.dtype}, max,min: {res_skimage.max()},{res_skimage.min()}")

Starting RL calculation using skimage.restoration.richardson_lucy()
Completed
res_skimage shape:(256, 256, 256), dtype: float32, max,min: 0.0023623835295438766,-2.325488976731571e-12


In [25]:
print("Starting RL calculation using doRLDeconvolution12()")
res3 = rls.doRLDeconvolution12(data_large,psf_norm, niter=3)
print("Completed")
print(f"res3 shape:{res3.shape}, dtype: {res3.dtype}, max,min: {res3.max()},{res3.min()}")

INFO:root:change3DSizeTo() , shape:[320, 288, 288]


INFO:root:change3DSizeTo() , shape:[320, 288, 288]


Starting RL calculation using doRLDeconvolution12()


INFO:root:change3DSizeTo() , shape:(256, 256, 256)


Completed
res3 shape:(256, 256, 256), dtype: float32, max,min: 0.010176570154726505,-1.5612606148351915e-05


In [26]:
#Maximum difference
print(np.max(np.abs(res0-res_ocl_cuda)))
print(np.max(np.abs(res0-res2)))
print(np.max(np.abs(res0-res2)))

0.00010056159
0.0013733553
0.0013733553


In [27]:
#Just for curiosity, compare the sum of values before and after convolution
print(np.sum(data_large))
print(np.sum(res0))
print(np.sum(res_ocl_cuda))
print(np.sum(res2))
print(np.sum(res_skimage))

512.0001
516.32837
516.5071
517.5692
517.5157


In [28]:
print(np.max(data_large), np.min(data_large))
print(np.max(res0), np.min(res0))
print(np.max(res_ocl_cuda), np.min(res_ocl_cuda))
print(np.max(res2), np.min(res2))
print(np.max(res_skimage), np.min(res_skimage))

0.0017753458 -3.2193867e-10
0.009700462 -8.700174e-06
0.00969618 0.0
0.009548041 0.0
0.0023623835 -2.325489e-12


In [29]:
contrastlimit0 = [0.0,0.01]
NV=napari.Viewer()
NV.add_image(data_large, contrast_limits=contrastlimit0)
NV.add_image(res0, contrast_limits=contrastlimit0)
NV.add_image(res_ocl_cuda, contrast_limits=contrastlimit0)
NV.add_image(res2, contrast_limits=contrastlimit0)
NV.add_image(res_skimage, contrast_limits=contrastlimit0)

<Image layer 'res_skimage' at 0x14710795c90>

In [30]:
#Maximum difference
print(np.max(np.abs(res_ocl_cuda-res0)))
print(np.max(np.abs(res2-res0)))
print(np.max(np.abs(res2-res_ocl_cuda)))
print(np.max(np.abs(res_skimage-res0)))

0.00010056159
0.0013733553
0.0013704346
0.007582476


In [31]:
#Check between each others
print(np.allclose(res_ocl_cuda,res0, rtol=1e-2, atol=1e-2))
print(np.allclose(res2,res0, rtol=1e-2, atol=1e-2))
print(np.allclose(res2,res_ocl_cuda, rtol=1e-2, atol=1e-2))
print(np.allclose(res_skimage,res0, rtol=1e-2, atol=1e-2))

True
True
True
True
